<a href="https://colab.research.google.com/github/jacobpad/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-01-16 02:54:15--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.184.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.184.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’

instacart_online_gr 100%[===================>] 196.03M  56.6MB/s    in 3.5s    

2020-01-16 02:54:19 (56.6 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
##### YOUR CODE HERE #####

In [0]:
# Pandas!
import pandas as pd

In [0]:
# Check. Do I need this?
# No - I'll comment it out

# aisles = pd.read_csv('aisles.csv')
# print(aisles.shape)
# aisles.head()

In [0]:
# Check. Do I need this?
# No - I'll comment it out

# departments = pd.read_csv('departments.csv')
# print(departments.shape)
# departments.head()

In [9]:
# Check. Do I need this?
order_products__prior = pd.read_csv('order_products__prior.csv')
print(order_products__prior.shape)
order_products__prior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [10]:
# Check. Do I need this?
order_products__train = pd.read_csv('order_products__train.csv')
print(order_products__train.shape)
order_products__train.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [11]:
# Clear the output of the above two tables, because I've combined them

# Concat the tables ([order_products__prior, order_products__train])
order_products = pd.concat([order_products__prior, order_products__train])
print(order_products.shape)
order_products.head()

(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [12]:
# Check. Do I need this?
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [13]:
# Check. Do I need this?
products = pd.read_csv('products.csv')
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [0]:
# How many times was each ordered? 
# 1. Banana
# 2. Bag of Organic Bananas
# 3. Organic Strawberries
# 4. Organic Baby Spinach 
# 5. Organic Hass Avocado
# 6. Organic Avocado
# 7. Large Lemon 
# 8. Strawberries
# 9. Limes 
# 10. Organic Whole Milk
# First, write down which columns you need and which dataframes have them.
# Next, merge these into a single dataframe.
# Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [15]:
# More merging
product_id_and_name = pd.merge(order_products, products, on='product_id', how='inner')
product_id_and_name

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16
...,...,...,...,...,...,...,...
33819101,2269288,27499,31,0,Non-Dairy Coconut Seven Layer Bar,100,21
33819102,2507248,27499,9,0,Non-Dairy Coconut Seven Layer Bar,100,21
33819103,2271346,37703,15,0,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...,75,17
33819104,3004244,43725,25,0,Sweetart Jelly Beans,100,21


In [16]:
# Remove the unneccasary columns
product_id_and_name = product_id_and_name.drop(['reordered','aisle_id', 'department_id'], axis=1)
product_id_and_name

,order_id,product_id,add_to_cart_order,product_name
0,2,33120,1,Organic Egg Whites
1,26,33120,5,Organic Egg Whites
2,120,33120,13,Organic Egg Whites
3,327,33120,5,Organic Egg Whites
4,390,33120,28,Organic Egg Whites
...,...,...,...,...
33819101,2269288,27499,31,Non-Dairy Coconut Seven Layer Bar
33819102,2507248,27499,9,Non-Dairy Coconut Seven Layer Bar
33819103,2271346,37703,15,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...
33819104,3004244,43725,25,Sweetart Jelly Beans


In [17]:
# Should give me the count of the number of products in the df 
# --> from here, take the number of times the order_id appears
pd.unique(product_id_and_name['product_id'].value_counts())

array([491291, 394930, 275577, ...,      3,      2,      1])

In [18]:
product_id_and_name['product_id'].value_counts().head(10)

24852    491291
13176    394930
21137    275577
21903    251705
47209    220877
47766    184224
47626    160792
16797    149445
26209    146660
27845    142813
Name: product_id, dtype: int64

In [19]:
top_ten = product_id_and_name['product_id'].value_counts().head(10)
top_ten

24852    491291
13176    394930
21137    275577
21903    251705
47209    220877
47766    184224
47626    160792
16797    149445
26209    146660
27845    142813
Name: product_id, dtype: int64

In [20]:
# Make it a df
may_be_final = pd.merge(product_id_and_name, top_ten, on='product_id', how='inner')
may_be_final

,order_id,product_id,add_to_cart_order,product_name


In [41]:
# Forget it - here's the top 10 with the quantity of each sold
top_10 = product_id_and_name['product_name'].value_counts().head(10)
top_10.to_frame()

,product_name
Banana,491291
Bag of Organic Bananas,394930
Organic Strawberries,275577
Organic Baby Spinach,251705
Organic Hass Avocado,220877
Organic Avocado,184224
Large Lemon,160792
Strawberries,149445
Limes,146660
Organic Whole Milk,142813


In [0]:
#### IGNORE THE FOLLOWING BUNCH OF WRONG CODE ####

In [0]:
# # merge
# new_df_1 = pd.merge(products, order_products__train, on='product_id', how='inner')
# new_df_1

In [0]:
# # Drop aisle_id	department_id	add_to_cart_order	reordered
# new_df_1 = new_df_1.drop(['aisle_id','department_id','add_to_cart_order','reordered'], axis=1)
# new_df_1

In [0]:
# # Drop
# order_products__prior = order_products__prior.drop(['add_to_cart_order','reordered'], axis=1)
# order_products__prior

In [0]:
# new_df_2 = pd.merge(new_df_1, order_products__prior, on='product_id', how='inner')
# new_df_2

In [0]:
# First, write down which columns you need and which dataframes have them.
# I want:
# -product name from products
# -product id from products
# -order id from orders
# -

In [0]:
# Just curious how many users there are
# orders['user_id'].value_counts().sum()

In [0]:
# Start a merge
# order_quantity_of_top_10 = pd.concat([products, orders])

In [0]:
# # Concat the tables in ()
# order_products = pd.concat([order_products__prior, order_products__train])
# print(order_products.shape)
# order_products.head()

In [0]:
# # Drop add_to_cart_order
# order_products = order_products.drop(['add_to_cart_order'], axis=1)
# print(order_products.shape)
# order_products.head()

In [0]:
# another_df = pd.concat([products, order_products])
# print(another_df.shape)
# another_df.head()

In [0]:
# # Drop reordered
# order_products = order_products.drop(['reordered'], axis=1)
# print(order_products.shape)
# order_products.head()

In [0]:
# # Concat the name
# df_add_name = pd.concat([order_products, products])
# print(df_add_name.shape)
# df_add_name.head()

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [33]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
# table2 -> tidy

In [0]:
# tidy -> table2

In [36]:
flights = sns.load_dataset('flights')

NameError: ignored

In [0]:
# Flights Pivot Table

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####